In [1]:
import numpy as np
from keras.models import Sequential, Model, load_model
import pandas as pd
import os

C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\nrx398\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  stacklevel=1)
Using TensorFlow backend.


In [2]:
save_root = 'DecompResult/'

if not os.path.exists(save_root):
    os.makedirs(save_root)

In [3]:
def load(path, path_model1, path_model2):
    
    Doublets = pd.read_csv(path, index_col = 0).values.T
    Doublets = Doublets / Doublets.sum(axis = 1).reshape((-1, 1)) * 1e6
    
    model1 = load_model(path_model1)
    model2 = load_model(path_model2)
    
    return Doublets, model1, model2

def Decompose(model1, model2, Doublets, cutoff1, cutoff2, main_name, save_root):

    #raw
    pred_1 = model1.predict(Doublets)
    pred_2 = model2.predict(Doublets)

    np.save(save_root + 'pred_raw_' + main_name + '_1.npy', pred_1)
    np.save(save_root + 'pred_raw_' + main_name + '_2.npy', pred_2)

    #max
    pred_1_max = np.argmax(pred_1, axis = 1)
    pred_2_max = np.argmax(pred_2, axis = 1)

    np.save(save_root + 'pred_max_' + main_name + '_1.npy', np.array(pred_1_max))
    np.save(save_root + 'pred_max_' + main_name + '_2.npy', np.array(pred_2_max))

    #0.01
    pred_1_cf = np.unique(np.where(pred_1>cutoff1)[1])
    pred_2_cf = np.unique(np.where(pred_2>cutoff2)[1])

    np.save(save_root + 'pred_' + str(cutoff1) + '_' + main_name + '_1.npy', np.array(pred_1_cf))
    np.save(save_root + 'pred_' + str(cutoff2) + '_' + main_name + '_2.npy', np.array(pred_2_cf))

In [4]:
DoubFile = 'umi/pcSeq_umi.csv'
path_model1 = 'DecompModel/Decomp_hep_LEC_1.h5'
path_model2 = 'DecompModel/Decomp_hep_LEC_2.h5'
Doublets, model1, model2 = load(DoubFile, path_model1, path_model2)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [7]:
Decompose(model1, model2, Doublets,
          cutoff1=0.01, cutoff2=0.01,
          main_name = 'hep_LEC', save_root = save_root)

In [6]:
Decompose(model1, model2, Doublets,
          cutoff1=0.005, cutoff2=0.005,
          main_name = 'hep_LEC', save_root = save_root)